<a href="https://colab.research.google.com/github/Elwing-Chou/tibame20251203/blob/main/tibame20260112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 成品1
import urllib.request as req
import bs4 as bs
import os
import json

# M.....html/meta.json
def get_page_data(url, dirname="."):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1766978298.A.90C.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)
    # print(html)

    uid_text = "-"
    board_text = "-"
    title_text = "-"
    date_time_text = "-"
    # "uid" "type":-1/0/1(需/->/推) "text"  "ipfatetime"
    pushes = []

    metas = html.find_all("span", {"class":"article-meta-value"})
    if len(metas) == 4:
        uid, board, title, date_time = metas
        uid_text = uid.get_text().strip()
        board_text = board.get_text().strip()
        title_text = title.get_text().strip()
        date_time_text = date_time.get_text().strip()

    pushes_meta = html.find_all("div", {"class":"push"})
    for p in pushes_meta:
        p_tag, p_uid, p_text, p_ipdatetime = p.find_all("span")
        p_tag_text = p_tag.get_text().strip()
        p_uid_text = p_uid.get_text().strip()
        p_text_text = p_text.get_text().strip().replace(": ", "", 1)
        p_ipdatetime_text = p_ipdatetime.get_text().strip()

        p_tag_conv = {
            "推":1,
            "→":0,
            "噓":-1
        }
        p_tag_text = p_tag_conv[p_tag_text]

        p_dict = {
            "tag":p_tag_text,
            "uid":p_uid_text,
            "text":p_text_text,
            "ipdatetime":p_ipdatetime_text
        }
        pushes.append(p_dict)

    # extract
    main_content = html.find("div", {"id":"main-content"})
    metas = main_content.find_all("div", {"class":"article-metaline"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("div", {"class":"article-metaline-right"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("div", {"class":"push"})
    for m in metas:
        m.extract()
    metas = main_content.find_all("span", {"class":"f2"})
    for m in metas:
        m_text = m.get_text()
        if "發信站:" in m_text:
            m.extract()
        if "文章網址:" in m_text:
            m.extract()

    main_content_text = main_content.get_text()

    data = {
        "uid":uid_text,
        "board":board_text,
        "title":title_text,
        "datetime":date_time_text,
        "content":main_content_text,
        "pushes":pushes
    }

    if not os.path.exists(dirname):
        os.makedirs(dirname)

    fp = dirname + "/meta.json"
    f = open(fp, "w", encoding="utf-8")
    json.dump(data, f, ensure_ascii=False, indent=4)
    f.close()

    return data

# 成品2
# .: 這層 ..: 上一層
def download_img(url, dirname="."):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    # url = "https://i.imgur.com/opGiGRV.jpeg"
    h = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
    }
    r = req.Request(url, headers=h)
    resp = req.urlopen(r)
    content = resp.read()
    # 1. 純文字: txt 2. 非純文字
    # r/w+encoding      rb/wb
    fp = dirname + "/" + url.split("/")[-1]
    # print(fp)
    f = open(fp, "wb")
    f.write(content)
    f.close()

# demo
# download_img("https://i.imgur.com/opGiGRV.jpeg", "aa")

In [11]:
def get_page_imgs(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1767140595.A.150.html"
    resp = req.urlopen(url)
    content = resp.read()
    html = bs.BeautifulSoup(content)

    save_dir = url.split("/")[-1]
    get_page_data(url, save_dir)

    links = html.find_all("a")
    allow_subnames = {"jpg", "png", "gif", "jpeg", "mp4"}
    for l in links:
        href = l["href"]
        sub = href.split(".")[-1]
        if sub.lower() in allow_subnames:
            print(href)
            download_img(href, save_dir)

In [12]:
import urllib.request as req
import bs4 as bs
url = "https://www.ptt.cc/bbs/Beauty/index4001.html"
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

titles = html.find_all("div", {"class":"title"})
for t in titles:
    t_link = t.find("a")
    if not t_link == None:
        t_link_href = "https://www.ptt.cc" + t_link["href"]
        t_link_text = t_link.get_text()
        print(t_link_href)
        print(t_link_text)
        if not "公告" in t_link_text:
            get_page_imgs(t_link_href)
            get_page_data(t_link_href)

https://www.ptt.cc/bbs/Beauty/M.1768138487.A.38D.html
[正妹] IVE 李瑞 Leeseo
https://i.meee.com.tw/NLKrusg.jpg
https://i.meee.com.tw/uk7jLzc.jpg
https://i.meee.com.tw/vm1gvw8.jpg
https://i.meee.com.tw/0uiBGz9.jpg
https://i.meee.com.tw/VqWhvrt.gif
https://i.meee.com.tw/kA8b1Gh.gif
https://i.meee.com.tw/Jem80dZ.gif
https://i.meee.com.tw/NW70WWZ.gif
https://www.ptt.cc/bbs/Beauty/M.1768153595.A.CE3.html
[正妹] 陳奶糖
https://i.imgur.com/USsogRa.jpeg
https://i.imgur.com/l8wh8xo.jpeg
https://i.imgur.com/kOYBN4Z.jpeg
https://i.imgur.com/SHcUmZu.jpeg
https://i.imgur.com/XShpvIi.jpeg
https://www.ptt.cc/bbs/Beauty/M.1768153719.A.439.html
[正妹] 許雅涵
https://i.imgur.com/eJ68FEL.jpeg
https://i.imgur.com/Ez197JI.jpeg
https://i.imgur.com/JeGUWDb.jpeg
https://i.imgur.com/fKUTcYJ.jpeg
https://i.imgur.com/v9Goe4i.jpeg
https://i.imgur.com/UgXnqC4.jpeg
https://www.ptt.cc/bbs/Beauty/M.1768153826.A.83B.html
[正妹] 思思
https://i.imgur.com/BpQIuVy.jpeg
https://i.imgur.com/5PlAF2b.jpeg
https://i.imgur.com/tawuLI5.jpeg
https